PCA-Based Drift Detection
1) Standardizzare l'intervallo dei valori delle feature iniziali per farle contribuire allo stesso modo all'analisi.
2) Calcolare la matrici di covarianza delle feature dei due data sets (Old and New).
3) Estrarre i vettori degli autovalori associati agli autovettori delle due matrici di covarianza.
4) Il grado di deriva tra l'insieme dei dati New e Old viene calcolato misurando l'angolo di intersezione tra gli autovalori dei due insiemi, cioà la varianza delle componenti principali dei due set di dati.
5) Se l'angolo è >= ALLARME && < DRIFT viene segnalato un allarme, se l'angolo è >= DRIFT viene segnalato un drift.

40° allarme, 50° drift.               in gradi d'arco
0,7 allarme, 0,875 drift.             in radianti

Problema angolo!

Con la distanza euclidea fra i vettori degli autovalori riesco a identificare il drift!

In [439]:
import pandas as pd
import numpy as np
import csv
import skmultiflow as sk
import sklearn.decomposition as dec
from sklearn.preprocessing import StandardScaler
import scipy.linalg as lin
size=150
dataset=pd.read_csv('SRdrift.csv' ,usecols=["A","B","C"])
data_stream= sk.data.DataStream(dataset ,y=None, n_targets=0)
data_stream.print_df()
data_stream.prepare_for_use()
 
data=data_stream.next_sample(batch_size=size)[0]
old_w=pd.DataFrame(data, columns=['A','B','C'])
print(f"Old:{old_w}")

[[-258.93763863 -654.19491326 -570.7438663 ]
 [-258.70523191 -655.15165221 -570.92558994]
 [-259.4375307  -653.76023233 -570.98461852]
 ...
 [-258.82138476 -654.07704964 -571.45474237]
 [-259.22645187 -654.94091298 -570.51667721]
 [-259.7429683  -654.39650835 -572.13457074]]
[]
Old:              A           B           C
0   -258.937639 -654.194913 -570.743866
1   -258.705232 -655.151652 -570.925590
2   -259.437531 -653.760232 -570.984619
3   -259.121990 -654.550492 -571.737737
4   -259.557844 -653.905345 -571.012381
..          ...         ...         ...
145 -258.798988 -654.872263 -571.819494
146 -259.473796 -654.991745 -571.522705
147 -259.305697 -654.621364 -571.014293
148 -258.883881 -654.640936 -571.346336
149 -259.249610 -654.842203 -571.043125

[150 rows x 3 columns]


c:\Users\Salvatore Drago\Desktop\progetto_lab\lib\site-packages\skmultiflow\data\base_stream.py:194: FutureWarning: 'prepare_for_use' has been deprecated in v0.5.0 and will be removed in v0.7.0.
New instances of the Stream class are now ready to use after instantiation.
  category=FutureWarning)


In [440]:
data=data_stream.next_sample(batch_size=size)[0]
new_w=pd.DataFrame(data, columns=['A','B','C'])
print(f"New:{new_w}")

New:               A            B           C
0    -259.333649  -653.984232 -570.601494
1   -1274.025129 -1070.408882 -672.300384
2    -259.467185  -653.903700 -571.505976
3   -1276.406205 -1071.262823 -671.739428
4    -258.980872  -653.946036 -570.967974
..           ...          ...         ...
145  -259.422103  -654.529905 -571.195691
146  -259.349803  -654.662150 -571.441323
147  -259.382980  -654.241537 -571.134050
148  -259.371850  -654.970870 -571.008178
149  -259.573782  -654.788964 -570.739765

[150 rows x 3 columns]


df=pd.merge(old_w, new_w, how='outer')
print(df)

#STANDARDIZZAZIONE
sc= StandardScaler()
sc.fit(df)
old_w_std=sc.transform(old_w)
new_w_std= sc.transform(new_w)


print(f"STD OLD: {old_w_std}")

print(f"STD NEW: {new_w_std}")

In [441]:
pca_old=dec.PCA(n_components=None, copy=True, whiten=False, svd_solver='auto')
pca_old.fit(old_w)
old_eigenvalues= pca_old.explained_variance_
print(f"OLD eigenvalues: {old_eigenvalues}")


OLD eigenvalues: [6.98529179e+04 1.26851200e+04 2.18518828e-01]


In [442]:
pca_new=dec.PCA(n_components=None, copy=True, whiten=False, svd_solver='auto')
pca_new.fit(new_w)
new_eigenvalues= pca_new.explained_variance_
print(f"NEW eigenvalues: {new_eigenvalues}")

NEW eigenvalues: [7.74168535e+04 4.98186685e+04 2.18285029e-01]


In [443]:
#v1=np.array([1,3])
#v2=np.array([4,2])
print(old_eigenvalues,new_eigenvalues)
print(np.dot(old_eigenvalues,new_eigenvalues))
print((np.dot(old_eigenvalues,new_eigenvalues))/((np.linalg.norm(old_eigenvalues)*np.linalg.norm(new_eigenvalues))))
angolo= np.arccos((np.dot(old_eigenvalues,new_eigenvalues))/((np.linalg.norm(old_eigenvalues)*np.linalg.norm(new_eigenvalues))))
print(angolo)

[6.98529179e+04 1.26851200e+04 2.18518828e-01] [7.74168535e+04 4.98186685e+04 2.18285029e-01]
6039748894.802311
0.9240854220014169
0.39216071664327656


In [444]:
sum= 0
for i in range (0, len(old_eigenvalues)):
    sum=sum+((old_eigenvalues[i]-new_eigenvalues[i])**2)
dist_euc= sum**(1/2)
print(f"Dist eucl: {dist_euc}")

Dist eucl: 37896.0887756549
